# Data Collection


In [9]:
import os 
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from tqdm import tqdm
import random
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

In [12]:
PDF_DIR = "niva_bupa_pdfs"

# Create directory to store PDFs
os.makedirs(PDF_DIR, exist_ok=True)
url = "https://www.nivabupa.com/downloads.html"


In [10]:
def get_soup_object(url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in background
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    # Set up Selenium WebDriver
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    driver.get(url)
    time.sleep(5)  # Allow page to load

    # Get page source
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit() 


    return soup

soup = get_soup_object(url)

In [11]:
soup

<html lang="en"><!-- Head tag Starts --><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="script-src 'self' assets.adobedtm.com 'unsafe-inline' 'unsafe-eval' https:" http-equiv="Content-Security-Policy"/>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="4kzjmIJ6rjLICVR5sn_mFhgWDOzfqA_XBc6G85T4vAk" name="google-site-verification"/>
<link href="/etc.clientlibs/nivabupa/clientlib-global/css/resources/images/niva-bupa-favicon.png" rel="icon" type="image/png"/>
<link href="https://assets.adobedtm.com/" rel="dns-prefetch"/>
<link href="https://www.googletagmanager.com/" rel="dns-prefetch"/>
<link href="https://www.google-analytics.com/" rel="dns-prefetch"/>
<link href="https://fonts.googleapis.com" rel="preconnect"/>
<link crossorigin="" href="https://fonts.gstatic.com" rel="preconnect"/>
<link href="https://fonts.googleapis.com/css2?family=Roboto:wght@300;400;500;600;700&amp;family=Rubik:wght@400;500;600;700&amp;

In [46]:
download_sources = soup.find_all("div", class_="dwnldCrdContainer")
download_sources

[<div class="dwnldCrdContainer">
 <a class="dwnloadCard-wrapr" href="/content/dam/nivabupa/PDF/GoActive/goactive-brochure.pdf" target="_blank">
 <div class="row">
 <div class="col-12">
 <h3 class="headerText">
                                         GoActive <br/> Brochure
                                     </h3>
 <!-- <img src="../../assets/Img/icon-pdf-big.jpg" /> -->
 <img alt="pdficon" src="/etc.clientlibs/nivabupa/clientlib-global/css/resources/images/icon-pdf-mobil.jpg" title="pdficon"/>
 </div>
 </div>
 <div class="row">
 <div class="col-12">
 <div class="contentPara">
 <p>Download the brochure to understand the health companion plan better</p>
 </div>
 </div>
 </div>
 </a>
 <a class="dwnloadCard-wrapr" href="/content/dam/nivabupa/PDF/GoActive/goactive-policy-document.pdf" target="_blank">
 <div class="row">
 <div class="col-12">
 <h3 class="headerText">
                                         GoActive Policy <br/> Wordings
                                     </h3>
 <!-- <i

In [27]:
len(download_sources)

28

In [28]:
pdf_links = []
BASE_URL = "https://www.nivabupa.com"
for section in download_sources:
    for a_tag in section.find_all(href=True):
        print(a_tag)
        pdf_url = a_tag["href"]
        if pdf_url.endswith(".pdf"):
            # Ensure absolute URL
            full_url = BASE_URL + pdf_url if pdf_url.startswith("/") else pdf_url
            pdf_links.append(full_url)

In [29]:
len(pdf_links)

0

In [17]:
with open("pdf_links.txt", "w") as f:
    for link in pdf_links:
        f.write(link + "\n")

print("PDF links saved to pdf_links.txt")

PDF links saved to pdf_links.txt


In [19]:
def download_pdfs(pdf_links):
   
    for url in tqdm(pdf_links, desc="Downloading PDFs"):
        pdf_name = url.split("/")[-1]
        pdf_path = os.path.join(PDF_DIR, pdf_name)

        # Skip if already downloaded
        if os.path.exists(pdf_path):
            print(f"Skipping {pdf_name}, already downloaded.")
            continue

        # Download the PDF
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(pdf_path, "wb") as f:
                for chunk in response.iter_content(1024):
                    f.write(chunk)
            print(f"Downloaded {pdf_name}")
        else:
            print(f"Failed to download {pdf_name}")

# Run the download process
download_pdfs(pdf_links)
print("All PDFs downloaded successfully.")

Skipping goactive-brochure.pdf, already downloaded.
Skipping goactive-policy-document.pdf, already downloaded.


Downloaded NivaBupa-pre-auth-claim-form.pdf


Downloaded GoActive-Proposal-Form.pdf


Downloaded heartbeat%20brochure.pdf


Downloaded policy%20wording.pdf


...

Downloaded nivabupa-health-pulse-t-and-c.pdf


Downloaded HP-Retail-AppForm.pdf


Downloaded healthpremia%20brochure.pdf


Downloaded Health%20Premia%20Policy%20Wording.pdf


Downloaded HPR-RETAIL-AppForm.pdf
All PDFs downloaded successfully.
